In [13]:
pip install pillow

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.7 MB 751.6 kB/s eta 0:00:04
   ---------- ----------------------------- 0.7/2.7 MB 4.3 MB/s eta 0:00:01
   ------------------- -------------------- 1.3/2.7 MB 6.5 MB/s eta 0:00:01
   ----------------------------- ---------- 2.0/2.7 MB 7.9 MB/s eta 0:00:01
   -------------------------------- ------- 2.2/2.7 MB 7.2 MB/s eta 0:00:01
   ---------------------------------------  2.7/2.7 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pyautogui
import time

time.sleep(5)
print(pyautogui.position())

Point(x=46, y=79)


In [5]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json

# List of news sources
news_sources = ['nytimes', 'washingtonpost', 'usatoday', 'wallstreetjournal', 'latimes', 'cnn', 'foxnews', 'msnbc', 'abcnews', 'cbsnews', 'nbcnews', 'pbsnews', 'huffpost']

# Launch TikTok app
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"
os.startfile(shortcut_path)

# Wait for the app to open
time.sleep(5)

for source in news_sources:
    # Navigate back to search before starting a new search
    pyautogui.click(x=40, y=155)  # Close any open video (click back button)
    time.sleep(2)
    pyautogui.click(x=141, y=160)  # Click on search bar (or click on search button)
    time.sleep(2)

    # Clear previous search
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.press('backspace')

    # Type the search term
    pyautogui.write(source, interval=0.1)

    # Press enter to search
    pyautogui.press("enter")

    # Wait for search results to load
    time.sleep(4)

    # Click on the profile or first result
    pyautogui.click(x=659, y=210)

    # Wait for the profile/video page to load
    time.sleep(4)

    # Starting coordinates
    video_x_start = 508
    video_x = video_x_start
    video_y = 897

    video_counter = 0  # Total videos scraped
    row_video_counter = 0  # How many videos scraped in current row

    # DataFrame to store all comments
    all_comments = pd.DataFrame()
    row_counter = 0  # Tracks how many rows deep we are
    videos_in_current_row = 0

    while True:
        # Click on a video
        pyautogui.click(x=video_x, y=video_y)
        time.sleep(5)

        # Move and click inside the comment area to focus it
        pyautogui.click(x=1605, y=591)
        time.sleep(1)

        # Scroll comments until end
        previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
        while True:
            pyautogui.scroll(-500)
            time.sleep(0.9)
            current = pyautogui.screenshot(region=(1200, 600, 100, 100))
            if current == previous:
                print("Reached the end of the scroll.")
                break
            previous = current

        # Wait a moment before injecting
        time.sleep(2)

        # Focus TikTok app window
        pyautogui.click(x=300, y=300)
        time.sleep(1)

        # Open DevTools (F12)
        pyautogui.hotkey('fn', 'f12')
        time.sleep(2)

        # Focus Console tab
        pyautogui.click(x=496, y=840)
        time.sleep(2)

        # Inject JavaScript (untouched)
        js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

// Try to get the username (handle starting with "@")
const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) {
    username = usernameLink.innerText.trim();
}

// Try to get the video caption
const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) {
    caption = captionBlock.innerText.trim();
}

// Try to get the video link
const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) {
    videoLink = videoLinkBlock.innerText.trim();
}

// Try to get the video date
const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) {
            // e.g., 2024-8-3
            videoDate = text;
        } else if (text.match(/^\d{1,2}-\d{1,2}/)) {
            // e.g., 8-3
            videoDate = text;
        } else if (text.match(/^\d+[mhdy]\s*ago$/i)) {
            // e.g., 3d ago
            videoDate = text;
        }
    }); // <-- this closes spans.forEach!
}

// Now collect comments
document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({
        comment,
        commentDate,
        likes,
        videoDate,
        username,
        caption,
        videoLink
    });
});

console.log(comments);
copy(comments);

"""

        pyperclip.copy(js_code)
        pyautogui.hotkey('ctrl', 'v')
        time.sleep(1)
        pyautogui.press('enter')

        # Wait for JS to execute and clipboard to update
        time.sleep(5)

        # Get clipboard content
        clipboard_content = pyperclip.paste()

        try:
            comments = json.loads(clipboard_content)
            df = pd.DataFrame(comments)
            all_comments = pd.concat([all_comments, df], ignore_index=True)

            # Check the video date to decide if we stop
            if not df.empty:
                video_date_sample = df['videoDate'].iloc[0]
                print("Video Date Found:", video_date_sample)

                if "-" in video_date_sample:
                    try:
                        if video_date_sample.count("-") == 2:
                            year, month, day = map(int, video_date_sample.split("-"))
                        elif video_date_sample.startswith("2024-"):
                            date_part = video_date_sample.split("2024-")[1]
                            month, day = map(int, date_part.split("-"))
                            year = 2024
                        else:
                            month, day = map(int, video_date_sample.split("-"))
                            year = 2025

                        if year == 2024:
                            if (month < 7) or (month == 7 and day < 1):
                                print(f"Video is from {year}-{month}-{day}, stopping scraping.")
                                break
                        elif year < 2024:
                            print(f"Video is from {year}-{month}-{day}, stopping scraping.")
                            break

                    except ValueError:
                        print("Date format error, skipping date check.")

        except Exception as e:
            print("Failed to parse clipboard content as JSON:", e)
            print("Clipboard content was:")
            print(clipboard_content)

        # Close video
        pyautogui.click(x=46, y=79)
        time.sleep(2)

        # After exiting, TikTok jumps to top, so we re-scroll down
        for _ in range(row_counter):
            pyautogui.scroll(-500)
            time.sleep(1)

        # Update counters
        video_counter += 1
        row_video_counter += 1
        videos_in_current_row += 1
        print(f"Scraped {video_counter} videos")

        # Move to next
        if row_video_counter == 4:
            row_counter += 1
            pyautogui.scroll(-500)
            time.sleep(1)

            video_x = video_x_start
            row_video_counter = 0
            videos_in_current_row = 0
        else:
            video_x += 500

    # Save all collected comments for this source
    all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
    print(f"All comments saved to tiktok_comments_scraped_{source}.csv")


<>:93: SyntaxWarning: invalid escape sequence '\d'
<>:93: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_11192\2059999044.py:93: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Reached the end of the scroll.
Video Date Found: 2h ago
Scraped 1 videos
Reached the end of the scroll.
Video Date Found: 23h ago
Scraped 2 videos
Reached the end of the scroll.
Video Date Found: 2d ago
Scraped 3 videos
Reached the end of the scroll.
Video Date Found: 3d ago
Scraped 4 videos
Reached the end of the scroll.
Video Date Found: 2d ago
Scraped 5 videos
Reached the end of the scroll.
Video Date Found: 2d ago
Scraped 6 videos
Reached the end of the scroll.
Video Date Found: 3d ago
Scraped 7 videos
Reached the end of the scroll.
Video Date Found: 4d ago
Scraped 8 videos
Reached the end of the scroll.
Video Date Found: 3d ago
Scraped 9 videos
Reached the end of the scroll.
Video Date Found: 4d ago
Scraped 10 videos
Reached the end of the scroll.
Video Date Found: 4d ago
Scraped 11 videos
Reached the end of the scroll.
Video Date Found: 5d ago
Scraped 12 videos
Reached the end of the scroll.
Video Date Found: 4d ago
Scraped 13 videos
Reached the end of the scroll.
Video Date Foun

C:\Users\zhout\AppData\Local\Temp\ipykernel_11192\2059999044.py:93: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [6]:
import pyautogui
import time

time.sleep(5)
print(pyautogui.position())

Point(x=45, y=81)


In [ ]:
#every video will always be in this order, use these exact coordinats in this order: 

#first video: 
Point(x=511, y=851)
#follow comment scrolling logic
#exit the video
#locate the "Just watched" button and click it
Point(x=1723, y=933) #will ALWAYS be these coordinates

#scrolls to last watched, first video
#second video:
Point(x=911, y=430)
#follow comment scrolling logic
#exit the video
#locate the "Just watched" button and click it
Point(x=1723, y=933) #will ALWAYS be these coordinates

#scrolls to last watched, second video
#third video: 
Point(x=1304, y=427)
#follow comment scrolling logic
#exit the video
#locate the "Just watched" button and click it
Point(x=1723, y=933) #will ALWAYS be these coordinates

#scrolls to last watched, third video
#fourth video: 
Point(x=1697, y=416)
#follow comment scrolling logic
#exit the video
#locate the "Just watched" button and click it
Point(x=1723, y=933) #will ALWAYS be these coordinates

#END OF ROW

#NOT IT WILL BE A LITTLE DIFFERENT
#Location of the 5th video: 
Point(x=503, y=886)
#follow comment scrolling logic
#exit the video
#locate the "Just watched" button and click it
Point(x=1723, y=933) #will ALWAYS be these coordinates
#NOW FOLLOW THE LOGIC FOR VIDEOS 2-4



In [13]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json

# ========== SETTINGS ==========
news_sources = ['nytimes', 'washingtonpost', 'usatoday', 'wallstreetjournal', 'latimes', 'cnn', 'foxnews', 'msnbc', 'abcnews', 'cbsnews', 'nbcnews', 'pbsnews', 'huffpost']
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Coordinates
video_click_points = [
    (511, 851),  # First video
    (911, 430),  # Second video
    (1304, 427), # Third video
    (1697, 416), # Fourth video
]
fifth_video_point = (503, 886)
just_watched_button = (1723, 933)

# ========== FUNCTIONS ==========

def scrape_current_video():
    """Scrape comments from the current video and return as a DataFrame."""
    # Move and click inside the comment area
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    # Scroll comments until end
    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-2000)
        time.sleep(1.5)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)

    # Focus app window
    pyautogui.click(x=300, y=300)
    time.sleep(1)

    # Open DevTools (F12)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)

    # Focus Console
    pyautogui.click(x=496, y=840)
    time.sleep(2)

    # Inject JS code
    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""
    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')

    # Wait for JS execution
    time.sleep(5)

    # Read clipboard
    clipboard_content = pyperclip.paste()

    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()

def is_video_too_old(video_date_str):
    """Check if video date is before July 1st, 2024."""
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2024-"):
                date_part = video_date_str.split("2024-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2024
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2024 or (year == 2024 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

# ========== MAIN LOGIC ==========

# Launch TikTok
os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    # Navigate back to search
    pyautogui.click(x=40, y=155)  # Back button
    time.sleep(2)
    pyautogui.click(x=141, y=160)  # Search bar
    time.sleep(2)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.press('backspace')
    pyautogui.write(source, interval=0.1)
    pyautogui.press('enter')
    time.sleep(4)
    pyautogui.click(x=659, y=210)  # Click first result
    time.sleep(4)

    all_comments = pd.DataFrame()
    video_counter = 0

    while True:
        # Handle 5th video logic
        if video_counter >= 4 and (video_counter % 4 == 0):
            pyautogui.click(x=fifth_video_point[0], y=fifth_video_point[1])
            time.sleep(5)
            df = scrape_current_video()

            if not df.empty:
                all_comments = pd.concat([all_comments, df], ignore_index=True)
                print(f"Scraped {len(df)} comments from video {video_counter + 1}")
                sample_date = df['videoDate'].iloc[0]
                if is_video_too_old(sample_date):
                    print(f"Video from {sample_date} is too old. Stopping.")
                    break

            pyautogui.click(x=46, y=79)  # Exit video
            time.sleep(2)
            pyautogui.click(x=just_watched_button[0], y=just_watched_button[1])
            time.sleep(3)
            video_counter += 1

        for point in video_click_points:
            pyautogui.click(x=point[0], y=point[1])
            time.sleep(5)
            df = scrape_current_video()

            if not df.empty:
                all_comments = pd.concat([all_comments, df], ignore_index=True)
                print(f"Scraped {len(df)} comments from video {video_counter + 1}")
                sample_date = df['videoDate'].iloc[0]
                if is_video_too_old(sample_date):
                    print(f"Video from {sample_date} is too old. Stopping.")
                    break

            pyautogui.click(x=46, y=79)
            time.sleep(2)
            pyautogui.click(x=just_watched_button[0], y=just_watched_button[1])
            time.sleep(3)
            video_counter += 1

    # Save all comments for this source
    all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
    print(f"Saved {len(all_comments)} total comments for {source}.")


<>:55: SyntaxWarning: invalid escape sequence '\d'
<>:55: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_11192\834861790.py:55: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Scraped 13 comments from video 1
Scraped 9 comments from video 2
Scraped 5 comments from video 3
Scraped 112 comments from video 4
Scraped 32 comments from video 5
Scraped 6 comments from video 6
Scraped 50 comments from video 7
Scraped 62 comments from video 8
Scraped 20 comments from video 9
Scraped 24 comments from video 10
Scraped 38 comments from video 11
Scraped 27 comments from video 12
Scraped 515 comments from video 13
Scraped 84 comments from video 14
Scraped 10 comments from video 15
Scraped 5 comments from video 16
Scraped 97 comments from video 17
Scraped 154 comments from video 18
Scraped 51 comments from video 19
Scraped 155 comments from video 20
Scraped 40 comments from video 21
Scraped 14 comments from video 22
Scraped 7 comments from video 23
Scraped 30 comments from video 24
Scraped 47 comments from video 25
Scraped 115 comments from video 26
Scraped 15 comments from video 27
Scraped 153 comments from video 28
Scraped 30 comments from video 29
Scraped 26 comments fr

C:\Users\zhout\AppData\Local\Temp\ipykernel_11192\834861790.py:55: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


KeyboardInterrupt: 

In [ ]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json

# ========== SETTINGS ==========


#news_sources = ['nytimes', 'washingtonpost', 'usatoday', 'wallstreetjournal', 'latimes', 'cnn', 'foxnews', 'msnbc', 'abcnews', 'cbsnews', 'nbcnews', 'pbsnews', 'huffpost']
#dev testing tools below 
news_sources = ['memezar', 'herrreff']
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Coordinates
first_video_point = (511, 851)  # Starting point (first video)
next_video_button = (1189, 527)  # Inside video, button to move to next

# ========== FUNCTIONS ==========

def scrape_current_video():
    """Scrape comments from the current video and return as a DataFrame."""
    # Move and click inside the comment area
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    # Scroll comments until end
    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(0.1)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)

    # Focus app window
    pyautogui.click(x=300, y=300)
    time.sleep(1)

    # Open DevTools (F12)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)

    # Focus Console
    pyautogui.click(x=496, y=840)
    time.sleep(2)

    # Inject JS code
    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""
    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')

    # Wait for JS execution
    time.sleep(5)

    # Read clipboard
    clipboard_content = pyperclip.paste()

    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()

def is_video_too_old(video_date_str):
    """Check if video date is before July 1st, 2024."""
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2024-"):
                date_part = video_date_str.split("2024-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2024
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2024 or (year == 2024 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

def restart_console():
    pyautogui.click(x=1539, y=132)  # Close DevTools
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=496, y=840)
    time.sleep(1)

# ========== MAIN LOGIC ==========

# Launch TikTok
os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    # Navigate back to search
    pyautogui.click(x=40, y=155)  # Back button
    time.sleep(2)
    pyautogui.click(x=141, y=160)  # Search bar
    time.sleep(2)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.press('backspace')
    pyautogui.write(source, interval=0.1)
    pyautogui.press('enter')
    time.sleep(4)
    pyautogui.click(x=659, y=210)  # Click first result
    time.sleep(4)

    all_comments = pd.DataFrame()
    video_counter = 0

    # Start by clicking the first video
    pyautogui.click(x=first_video_point[0], y=first_video_point[1])
    time.sleep(5)

    while True:
        df = scrape_current_video()

        if not df.empty:
            all_comments = pd.concat([all_comments, df], ignore_index=True)
            print(f"Scraped {len(df)} comments from video {video_counter + 1}")
            sample_date = df['videoDate'].iloc[0]
            if is_video_too_old(sample_date):
                print(f"Video from {sample_date} is too old. Stopping for {source}.")

                # Close the tab (profile)
                pyautogui.click(x=45, y=81)
                time.sleep(2)

                break
            
        #restart devtools to prevent overload
        restart_console()


        # Click "Next Video"
        pyautogui.click(x=1826, y=793) #click outside of the console screen
        time.sleep(1) #don't rush
        pyautogui.click(x=next_video_button[0], y=next_video_button[1])
        time.sleep(5)
        video_counter += 1

    # Save all comments for this source
    all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
    print(f"Saved {len(all_comments)} total comments for {source}.")


<>:53: SyntaxWarning: invalid escape sequence '\d'
<>:53: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_12660\828609981.py:53: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Scraped 41 comments from video 1
Scraped 39 comments from video 2
Scraped 36 comments from video 3
Video from 2023-11-29 is too old. Stopping for memezar.
Saved 116 total comments for memezar.
Scraped 42 comments from video 1
Scraped 37 comments from video 2
Video from 2024-6-16 is too old. Stopping for herrreff.
Saved 79 total comments for herrreff.


In [ ]:
import pyautogui
import time

time.sleep(5)
print(pyautogui.position())

Point(x=1887, y=12)


In [8]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json

# ========== SETTINGS ==========

# news_sources = ['nytimes', 'washingtonpost', ...]  # Full list for production
news_sources = ['memezar', 'herrreff']  # Dev testing
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Coordinates
first_video_point = (511, 851)  # First video
next_video_button = (1189, 527)  # Next video
console_close_button = (1539, 132)
console_console_tab = (496, 840)
search_back_button = (40, 155)
search_bar = (141, 160)
search_result_first = (659, 210)
exit_profile_button = (45, 81)
click_outside_console = (1826, 793)

# ========== FUNCTIONS ==========

def scrape_current_video():
    """Scrape comments from the current video and return as a DataFrame."""
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(0.1)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)
    pyautogui.click(x=300, y=300)
    time.sleep(1)

    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=console_console_tab[0], y=console_console_tab[1])
    time.sleep(2)

    js_code = """
    
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""  

    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')

    time.sleep(5)

    clipboard_content = pyperclip.paste()
    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()

def is_video_too_old(video_date_str):
    """Check if video date is before July 1st, 2024."""
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2024-"):
                date_part = video_date_str.split("2024-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2024
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2024 or (year == 2024 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

def restart_console():
    pyautogui.click(x=console_close_button[0], y=console_close_button[1])
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=console_console_tab[0], y=console_console_tab[1])
    time.sleep(1)

def restart_tiktok_app():
    """Completely close and reopen TikTok app."""
    os.system("taskkill /im TikTok.exe /f")  # Force-close TikTok (adjust EXE name if needed)
    time.sleep(5)
    os.startfile(shortcut_path)  # Reopen TikTok
    time.sleep(7)

# ========== MAIN LOGIC ==========

os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    last_collected_date = None
    all_comments = pd.DataFrame()

    def open_user_profile(source):
        """Navigate search and open profile."""
        pyautogui.click(x=search_back_button[0], y=search_back_button[1])
        time.sleep(2)
        pyautogui.click(x=search_bar[0], y=search_bar[1])
        time.sleep(2)
        pyautogui.hotkey('ctrl', 'a')
        pyautogui.press('backspace')
        pyautogui.write(source, interval=0.1)
        pyautogui.press('enter')
        time.sleep(4)
        pyautogui.click(x=search_result_first[0], y=search_result_first[1])
        time.sleep(4)

    open_user_profile(source)
    pyautogui.click(x=first_video_point[0], y=first_video_point[1])
    time.sleep(5)

    while True:
        df = scrape_current_video()

        if not df.empty:
            all_comments = pd.concat([all_comments, df], ignore_index=True)
            sample_date = df['videoDate'].iloc[0]
            print(f"Scraped {len(df)} comments. Video date: {sample_date}")

            # Set last collected date
            if last_collected_date is None:
                last_collected_date = sample_date

            # Check if date too old
            if is_video_too_old(sample_date):
                print(f"Reached video too old: {sample_date}")
                break

        restart_console()

        pyautogui.click(x=click_outside_console[0], y=click_outside_console[1])
        time.sleep(1)
        pyautogui.click(x=next_video_button[0], y=next_video_button[1])
        time.sleep(5)

    # Save intermediate progress
    all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
    print(f"Saved {len(all_comments)} comments for {source}.")

    # After batch: restart app
    print("Restarting TikTok App...")
    restart_tiktok_app()

    # After app restarts: open same user profile
    open_user_profile(source)
    pyautogui.click(x=first_video_point[0], y=first_video_point[1])
    time.sleep(5)

    # Scrape until seeing same/older video again
    while True:
        df = scrape_current_video()

        if not df.empty:
            video_date = df['videoDate'].iloc[0]
            print(f"Checking video date: {video_date}")

            if video_date <= last_collected_date:
                print(f"Already scraped up to {video_date}. Done with {source}.")
                break

        restart_console()

        pyautogui.click(x=click_outside_console[0], y=click_outside_console[1])
        time.sleep(1)
        pyautogui.click(x=next_video_button[0], y=next_video_button[1])
        time.sleep(5)

    print(f"Finished re-scraping after restart for {source}.")



<>:50: SyntaxWarning: invalid escape sequence '\d'
<>:50: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_12660\2489765184.py:50: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Scraped 41 comments. Video date: 2024-12-30
Scraped 39 comments. Video date: 2024-12-18
Scraped 36 comments. Video date: 2023-11-29
Reached video too old: 2023-11-29
Saved 116 comments for memezar.
Restarting TikTok App...


C:\Users\zhout\AppData\Local\Temp\ipykernel_12660\2489765184.py:50: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


KeyboardInterrupt: 

In [11]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pytesseract
from PIL import ImageGrab

In [ ]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json
import pytesseract
from PIL import ImageGrab

# ========== SETTINGS ==========

news_sources = ['memezar', 'herrreff']  # Dev testing
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Set your local Tesseract-OCR path
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\zhout\Desktop\HARDCORE_PYTHON\Python_Exercises\Pytesseract\tesseract.exe"

# Coordinates
first_video_point = (511, 851)  # First video
next_video_button = (1189, 527)  # Next video
console_close_button = (1539, 132)
console_console_tab = (496, 840)
search_back_button = (40, 155)
search_bar = (141, 160)
search_result_first = (659, 210)
exit_profile_button = (45, 81)
click_outside_console = (1826, 793)

# Region for reading video date (around Point(1489, 128))
ocr_date_region = (1450, 100, 1550, 160)  # (left, top, right, bottom)

# ========== FUNCTIONS ==========

def scrape_current_video():
    """Scrape comments from the current video and return as a DataFrame."""
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(0.1)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)
    pyautogui.click(x=300, y=300)
    time.sleep(1)

    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=console_console_tab[0], y=console_console_tab[1])
    time.sleep(2)

    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""  

    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')

    time.sleep(5)

    clipboard_content = pyperclip.paste()
    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()

def is_video_too_old(video_date_str):
    """Check if video date is before July 1st, 2024."""
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2024-"):
                date_part = video_date_str.split("2024-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2024
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2024 or (year == 2024 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

def restart_console():
    pyautogui.click(x=console_close_button[0], y=console_close_button[1])
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=console_console_tab[0], y=console_console_tab[1])
    time.sleep(1)

def restart_tiktok_app():
    """Close TikTok window by clicking 'X' and reopen TikTok app."""
    pyautogui.click(x=1887, y=12)  # Click to close the app window
    time.sleep(5)  # Give system a little time
    os.startfile(shortcut_path)  # Reopen the TikTok shortcut
    time.sleep(7)  # Wait for TikTok to fully open


def open_user_profile(source):
    """Navigate search and open profile."""
    pyautogui.click(x=search_back_button[0], y=search_back_button[1])
    time.sleep(2)
    pyautogui.click(x=search_bar[0], y=search_bar[1])
    time.sleep(2)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.press('backspace')
    pyautogui.write(source, interval=0.1)
    pyautogui.press('enter')
    time.sleep(4)
    pyautogui.click(x=search_result_first[0], y=search_result_first[1])
    time.sleep(4)

def read_visible_video_date():
    """Use OCR to read the visible video date shown on screen."""
    img = ImageGrab.grab(bbox=ocr_date_region)  # (left, top, right, bottom)
    text = pytesseract.image_to_string(img)
    text = text.strip().replace('\n', '').replace('\r', '')
    print(f"OCR extracted date text: '{text}'")
    return text

# ========== MAIN LOGIC ==========

os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    last_collected_date = None
    all_comments = pd.DataFrame()

    open_user_profile(source)
    pyautogui.click(x=first_video_point[0], y=first_video_point[1])
    time.sleep(5)

    # ========== Scrape normally until stopping ==========

    while True:
        df = scrape_current_video()

        if not df.empty:
            all_comments = pd.concat([all_comments, df], ignore_index=True)
            sample_date = df['videoDate'].iloc[0]
            print(f"Scraped {len(df)} comments. Video date: {sample_date}")

            if last_collected_date is None:
                last_collected_date = sample_date

            if is_video_too_old(sample_date):
                print(f"Reached video too old: {sample_date}")
                break

        restart_console()
        pyautogui.click(x=click_outside_console[0], y=click_outside_console[1])
        time.sleep(1)
        pyautogui.click(x=next_video_button[0], y=next_video_button[1])
        time.sleep(5)

    # Save progress
    all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
    print(f"Saved {len(all_comments)} comments for {source}.")

    # ========== Restart TikTok app ==========

    print("Restarting TikTok App...")
    restart_tiktok_app()
    open_user_profile(source)
    pyautogui.click(x=first_video_point[0], y=first_video_point[1])
    time.sleep(5)

    # ========== Wait until a different date appears visually ==========

    while True:
        current_visible_date = read_visible_video_date()
        if current_visible_date != last_collected_date:
            print(f"New video detected: {current_visible_date}. Resuming scraping.")
            break
        else:
            print(f"Same date ({current_visible_date}). Clicking next...")
            pyautogui.click(x=next_video_button[0], y=next_video_button[1])
            time.sleep(5)

    # ========== Resume scraping ==========
    while True:
        df = scrape_current_video()

        if not df.empty:
            video_date = df['videoDate'].iloc[0]
            print(f"Checking scraped video date: {video_date}")

            if video_date <= last_collected_date:
                print(f"Already scraped up to {video_date}. Done with {source}.")
                break

            all_comments = pd.concat([all_comments, df], ignore_index=True)

        restart_console()
        pyautogui.click(x=click_outside_console[0], y=click_outside_console[1])
        time.sleep(1)
        pyautogui.click(x=next_video_button[0], y=next_video_button[1])
        time.sleep(5)

    # Save final full dataset
    all_comments.to_csv(f"tiktok_comments_scraped_{source}_final.csv", index=False)
    print(f"Final saved all comments for {source}.")


<>:57: SyntaxWarning: invalid escape sequence '\d'
<>:57: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_12660\684850316.py:57: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Scraped 40 comments. Video date: 2024-12-30
Scraped 39 comments. Video date: 2024-12-18
Scraped 36 comments. Video date: 2023-11-29
Reached video too old: 2023-11-29
Saved 115 comments for memezar.
Restarting TikTok App...
OCR extracted date text: 'ww024-12-30'
New video detected: ww024-12-30. Resuming scraping.
Checking scraped video date: 2024-12-30
Already scraped up to 2024-12-30. Done with memezar.
Final saved all comments for memezar.


FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [21]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json
import pytesseract
from PIL import ImageGrab, ImageEnhance
import re

# ========== SETTINGS ==========

news_sources = ['memezar', 'herrreff']  # Dev testing
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Set your Tesseract OCR path
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\zhout\Desktop\HARDCORE_PYTHON\Python_Exercises\Pytesseract\tesseract.exe"

# Coordinates
first_video_point = (511, 851)
next_video_button = (1189, 527)
console_close_button = (1539, 132)
console_console_tab = (496, 840)
search_back_button = (40, 155)
search_bar = (141, 160)
search_result_first = (659, 210)
exit_profile_button = (45, 81)
click_outside_console = (1826, 793)

# OCR region for reading video date
ocr_date_region = (1450, 100, 1550, 160)

# ========== FUNCTIONS ==========

def preprocess_image(img):
    img = img.convert('L')
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2.0)
    return img

def clean_ocr_text(text):
    text = text.strip().replace(' ', '').replace('–', '-').replace('—', '-')
    text = re.sub(r'[^0-9\-/]', '', text)
    if text.startswith("0") and len(text) >= 8:
        text = "2" + text[1:]
    return text

def read_visible_video_date():
    img = ImageGrab.grab(bbox=ocr_date_region)
    img = preprocess_image(img)
    text = pytesseract.image_to_string(img, config='--psm 7 -c tessedit_char_whitelist=0123456789-/')
    text = clean_ocr_text(text)
    print(f"OCR extracted date text: '{text}'")
    return text

def scrape_current_video():
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(0.1)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)
    pyautogui.click(x=300, y=300)
    time.sleep(1)

    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=console_console_tab[0], y=console_console_tab[1])
    time.sleep(2)

    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""
    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')

    time.sleep(5)

    clipboard_content = pyperclip.paste()
    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()

def is_video_too_old(video_date_str):
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2024-"):
                date_part = video_date_str.split("2024-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2024
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2024 or (year == 2024 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

def restart_console():
    pyautogui.click(x=console_close_button[0], y=console_close_button[1])
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=console_console_tab[0], y=console_console_tab[1])
    time.sleep(1)

def restart_tiktok_app():
    pyautogui.click(x=1887, y=12)
    time.sleep(5)
    os.startfile(shortcut_path)
    time.sleep(7)

def open_user_profile(source):
    pyautogui.click(x=search_back_button[0], y=search_back_button[1])
    time.sleep(2)
    pyautogui.click(x=search_bar[0], y=search_bar[1])
    time.sleep(2)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.press('backspace')
    pyautogui.write(source, interval=0.1)
    pyautogui.press('enter')
    time.sleep(4)
    pyautogui.click(x=search_result_first[0], y=search_result_first[1])
    time.sleep(4)

# ========== MAIN LOGIC ==========

os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    move_to_next_source = False
    last_collected_date = None
    all_comments = pd.DataFrame()

    open_user_profile(source)
    pyautogui.click(x=first_video_point[0], y=first_video_point[1])
    time.sleep(5)

    # OCR check before starting scraping
    while True:
        current_visible_date = read_visible_video_date()

        if not current_visible_date:
            print("No date detected. Clicking next...")
            pyautogui.click(x=next_video_button[0], y=next_video_button[1])
            time.sleep(5)
            continue

        if is_video_too_old(current_visible_date):
            print(f"Visible video too old ({current_visible_date}), skipping {source}.")
            move_to_next_source = True
            break

        if last_collected_date and current_visible_date <= last_collected_date:
            print(f"OCR detected old/same date ({current_visible_date}). Clicking next...")
            pyautogui.click(x=next_video_button[0], y=next_video_button[1])
            time.sleep(5)
        else:
            print(f"New video detected ({current_visible_date}). Start scraping!")
            move_to_next_source = False
            break

    if move_to_next_source:
        print(f"Moving to next source after {source}.")
        continue

    # Start scraping after OCR confirmed
    while True:
        df = scrape_current_video()

        if not df.empty:
            video_date = df['videoDate'].iloc[0]
            print(f"Checking scraped video date: {video_date}")

            if is_video_too_old(video_date):
                print(f"Scraped video too old ({video_date}), moving to next source.")
                break

            if last_collected_date and video_date <= last_collected_date:
                print(f"Already scraped up to {video_date}. Done with {source}.")
                break

            all_comments = pd.concat([all_comments, df], ignore_index=True)

        restart_console()
        pyautogui.click(x=click_outside_console[0], y=click_outside_console[1])
        time.sleep(1)
        pyautogui.click(x=next_video_button[0], y=next_video_button[1])
        time.sleep(5)

    if not all_comments.empty:
        all_comments.to_csv(f"tiktok_comments_scraped_{source}_final.csv", index=False)
        print(f"Final saved all comments for {source}.")
    else:
        print(f"No comments scraped for {source}.")

    print(f"Done with {source}. Moving on...\n\n")


<>:78: SyntaxWarning: invalid escape sequence '\d'
<>:78: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_12660\1936015387.py:78: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


OCR extracted date text: '2924-12-30'
New video detected (2924-12-30). Start scraping!
Checking scraped video date: 2024-12-30
Checking scraped video date: 2024-12-18
Checking scraped video date: 2023-11-29
Scraped video too old (2023-11-29), moving to next source.
Final saved all comments for memezar.
Done with memezar. Moving on...


OCR extracted date text: ''
No date detected. Clicking next...
OCR extracted date text: ''
No date detected. Clicking next...


C:\Users\zhout\AppData\Local\Temp\ipykernel_12660\1936015387.py:78: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


KeyboardInterrupt: 

In [ ]:
import pyautogui
import time

time.sleep(5)
print(pyautogui.position())


Point(x=1887, y=12)


In [ ]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json

# ========== SETTINGS ==========


#news_sources = ['nytimes', 'washingtonpost', 'usatoday', 'wallstreetjournal', 'latimes', 'cnn', 'foxnews', 'msnbc', 'abcnews', 'cbsnews', 'nbcnews', 'pbsnews', 'huffpost']
#dev testing tools below 
news_sources = ['washingtonpost']
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Coordinates
first_video_point = (511, 851)  # Starting point (first video)
next_video_button = (1189, 527)  # Inside video, button to move to next

# ========== FUNCTIONS ==========

def scrape_current_video():
    """Scrape comments from the current video and return as a DataFrame."""
    # Move and click inside the comment area
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    # Scroll comments until end
    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(0.1)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)

    # Focus app window
    pyautogui.click(x=300, y=300)
    time.sleep(1)

    # Open DevTools (F12)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)

    # Focus Console
    pyautogui.click(x=496, y=840)
    time.sleep(2)

    # Inject JS code
    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""
    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')

    # Wait for JS execution
    time.sleep(5)

    # Read clipboard
    clipboard_content = pyperclip.paste()

    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()

def is_video_too_old(video_date_str):
    """Check if video date is before July 1st, 2024."""
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2024-"):
                date_part = video_date_str.split("2024-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2024
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2024 or (year == 2024 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

def restart_console():
    pyautogui.click(x=1539, y=132)  # Close DevTools
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=496, y=840)
    time.sleep(1)

# ========== MAIN LOGIC ==========

# Launch TikTok
os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    # Navigate back to search
    pyautogui.click(x=40, y=155)  # Back button
    time.sleep(2)
    pyautogui.click(x=141, y=160)  # Search bar
    time.sleep(2)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.press('backspace')
    pyautogui.write(source, interval=0.1)
    pyautogui.press('enter')
    time.sleep(4)
    pyautogui.click(x=659, y=210)  # Click first result
    time.sleep(4)

    all_comments = pd.DataFrame()
    video_counter = 0

    # Start by clicking the first video
    pyautogui.click(x=first_video_point[0], y=first_video_point[1])
    time.sleep(5)

    while True:
        df = scrape_current_video()

        if not df.empty:
            all_comments = pd.concat([all_comments, df], ignore_index=True)
            print(f"Scraped {len(df)} comments from video {video_counter + 1}")
            sample_date = df['videoDate'].iloc[0]
            if is_video_too_old(sample_date):
                print(f"Video from {sample_date} is too old. Stopping for {source}.")

                # Close the tab (profile)
                pyautogui.click(x=45, y=81)
                time.sleep(2)

                break
            
        #restart devtools to prevent overload
        restart_console()


        # Click "Next Video"
        pyautogui.click(x=1826, y=793) #click outside of the console screen
        time.sleep(1) #don't rush
        pyautogui.click(x=next_video_button[0], y=next_video_button[1])
        time.sleep(5)
        video_counter += 1

    # Save all comments for this source
    all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
    print(f"Saved {len(all_comments)} total comments for {source}.")


In [9]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json

# ========== SETTINGS ==========

news_sources = ['.9.krm', 'herrreff']
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Coordinates
first_video_point = (511, 851)
next_video_button = (1189, 527)
exit_tiktok_button = (1887, 12)

# ========== FUNCTIONS ==========


def scrape_current_video():
    pyautogui.click(x=1605, y=591)
    time.sleep(1)
    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(0.1)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current
    time.sleep(2)
    pyautogui.click(x=300, y=300)
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=496, y=840)
    time.sleep(2)

    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""
    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')
    time.sleep(5)

    clipboard_content = pyperclip.paste()
    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()


def get_video_month(video_date_str):
    try:
        if video_date_str.count('-') == 2:
            year, month, _ = map(int, video_date_str.split('-'))
        else:
            year = 2024
            month, _ = map(int, video_date_str.split('-'))
        return f"{year}-{month:02d}"
    except:
        return "unknown"


def restart_console():
    pyautogui.click(x=1539, y=132)
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=496, y=840)
    time.sleep(1)


def reopen_tiktok():
    os.startfile(shortcut_path)
    time.sleep(5)

# ========== MAIN LOGIC ==========

for source in news_sources:
    reopen_tiktok()
    pyautogui.click(x=40, y=155)
    time.sleep(2)
    pyautogui.click(x=141, y=160)
    time.sleep(2)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.press('backspace')
    pyautogui.write(source, interval=0.1)
    pyautogui.press('enter')
    time.sleep(4)
    pyautogui.click(x=659, y=210)
    time.sleep(4)
    pyautogui.click(x=first_video_point[0], y=first_video_point[1])
    time.sleep(5)

    click_counter = 0
    current_month = None
    all_comments = pd.DataFrame()

    while True:
        df = scrape_current_video()

        if df.empty:
            break

        sample_date = df['videoDate'].iloc[0]
        month = get_video_month(sample_date)

        if current_month is None:
            current_month = month

        if month != current_month:
            # Save file for previous month
            all_comments.to_csv(f"tiktok_comments_{source}_{current_month}.csv", index=False)
            print(f"Saved {len(all_comments)} comments for {source} - {current_month}")
            # Exit TikTok
            pyautogui.click(exit_tiktok_button[0], exit_tiktok_button[1])
            time.sleep(3)
            # Reopen and resume
            reopen_tiktok()
            pyautogui.click(x=40, y=155)
            time.sleep(2)
            pyautogui.click(x=141, y=160)
            time.sleep(2)
            pyautogui.hotkey('ctrl', 'a')
            pyautogui.press('backspace')
            pyautogui.write(source, interval=0.1)
            pyautogui.press('enter')
            time.sleep(4)
            pyautogui.click(x=659, y=210)
            time.sleep(4)
            pyautogui.click(x=first_video_point[0], y=first_video_point[1])
            time.sleep(5)
            # Resume by clicking "next" click_counter times
            for _ in range(click_counter):
                pyautogui.click(x=next_video_button[0], y=next_video_button[1])
                time.sleep(5)

            all_comments = pd.DataFrame()
            current_month = month
            click_counter = 0

        all_comments = pd.concat([all_comments, df], ignore_index=True)
        click_counter += 1

        restart_console()

        pyautogui.click(x=1826, y=793)
        time.sleep(1)
        pyautogui.click(x=next_video_button[0], y=next_video_button[1])
        time.sleep(5)

    # Final save
    if not all_comments.empty:
        all_comments.to_csv(f"tiktok_comments_{source}_{current_month}.csv", index=False)
        print(f"Saved {len(all_comments)} comments for {source} - {current_month}")

<>:40: SyntaxWarning: invalid escape sequence '\d'
<>:40: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_21804\2776946928.py:40: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Saved 41 comments for .9.krm - unknown
Saved 40 comments for .9.krm - 2024-04
Saved 80 comments for .9.krm - 2024-03


C:\Users\zhout\AppData\Local\Temp\ipykernel_21804\2776946928.py:40: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


KeyboardInterrupt: 

In [3]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   -- ------------------------------------- 0.6/9.4 MB 12.2 MB/s eta 0:00:01
   ----- ---------------------------------- 1.2/9.4 MB 13.1 MB/s eta 0:00:01
   ------- -------------------------------- 1.8/9.4 MB 14.6 MB/s eta 0:00:01
   ---------- ----------------------------- 2.5/9.4 MB 13.1 MB/s eta 0:00:01
   ------------ --------------------------- 3.0/9.4 MB 13.7 MB/s eta 0:00:01
   ---------------- ----------------------- 3.8/9.4 MB 13.4 MB/s eta 0:00:01
   ------------------ --------------------- 4.4/9.4 MB 13.5 MB/s eta 0:00:01
   --------------------- ------------------ 5.0/9.4 MB 13.2 MB/s eta 0:00:01
   ------------------------ --------------- 5.6/9.4 MB 13.8 MB/s eta 0:00:01
   -------------------------- ------------- 6.1/9.4 MB 14.0 MB/s eta 0:00:01
   ---------------------------- ----------- 6.7/9.4 MB 13.4 MB/s eta 0:00:01
   ------------------------------- -------- 7.3/9.4 MB 13.3 MB/s eta 0:00:01
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import json

In [7]:
#gave up on the tik tok app, comment sections were weird
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import json

# Configure ChromeDriver
options = Options()
options.add_argument('--disable-notifications')
options.add_argument('--start-maximized')

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 20)
actions = ActionChains(driver)

# Navigate to TikTok
driver.get('https://www.tiktok.com/?lang=en')
time.sleep(10)  # Wait for initial load

# Perform search
search_button = wait.until(EC.element_to_be_clickable((
    By.XPATH, "//div[contains(@class, 'DivIconWithRedDotContainer') and .//svg[contains(@viewBox, '0 0 48 48')]]"
)))
driver.execute_script("arguments[0].click();", search_button)

search_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[data-e2e='search-user-input']")))
search_input.send_keys('nytimes')
search_input.send_keys(Keys.RETURN)

# Click on profile
profile_link = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href*='/@nytimes']")))
profile_link.click()

# Click on the first video
first_video = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div[data-e2e='user-post-item']")))
first_video.click()

# Function to extract comments
def scrape_comments():
    comments_data = []

    # Scroll comment section
    comment_section = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.css-h5r7le-DivTabMenuWrapper")))
    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(2)

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    comments = driver.find_elements(By.CSS_SELECTOR, "div[class*='DivCommentContentContainer']")

    for comment_el in comments:
        comment_text = comment_el.find_element(By.CSS_SELECTOR, 'p[data-e2e="comment-level-1"]').text
        comment_time = comment_el.find_element(By.CSS_SELECTOR, '[data-e2e^="comment-time"]').text
        likes = comment_el.find_element(By.CSS_SELECTOR, '[data-e2e="comment-like-count"]').text
        comments_data.append({"comment": comment_text, "comment_time": comment_time, "likes": likes})

    return comments_data

# Scrape data from the video
video_comments = scrape_comments()
df = pd.DataFrame(video_comments)

# Save to CSV
df.to_csv('tiktok_comments.csv', index=False)
print("Saved comments to CSV")

# Navigate to next video
next_video_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-e2e='arrow-right']")))
next_video_btn.click()

# Clean up
time.sleep(5)
driver.quit()


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7EFA8C6A5+28789]
	(No symbol) [0x00007FF7EF9F5B20]
	(No symbol) [0x00007FF7EF888F9A]
	(No symbol) [0x00007FF7EF8DF346]
	(No symbol) [0x00007FF7EF8DF57C]
	(No symbol) [0x00007FF7EF932B17]
	(No symbol) [0x00007FF7EF90736F]
	(No symbol) [0x00007FF7EF92F7E3]
	(No symbol) [0x00007FF7EF907103]
	(No symbol) [0x00007FF7EF8CFFC0]
	(No symbol) [0x00007FF7EF8D1273]
	GetHandleVerifier [0x00007FF7EFDD1AED+3458237]
	GetHandleVerifier [0x00007FF7EFDE829C+3550316]
	GetHandleVerifier [0x00007FF7EFDDDB9D+3507565]
	GetHandleVerifier [0x00007FF7EFB52C6A+841274]
	(No symbol) [0x00007FF7EFA009EF]
	(No symbol) [0x00007FF7EF9FCB34]
	(No symbol) [0x00007FF7EF9FCCD6]
	(No symbol) [0x00007FF7EF9EC119]
	BaseThreadInitThunk [0x00007FF9DAFEE8D7+23]
	RtlUserThreadStart [0x00007FF9DB6B14FC+44]


In [1]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json

# ========== SETTINGS ==========


#news_sources = ['nytimes', 'washingtonpost', 'usatoday', 'wallstreetjournal', 'latimes', 'cnn', 'foxnews', 'msnbc', 'abcnews', 'cbsnews', 'nbcnews', 'pbsnews', 'huffpost']
#dev testing tools below 
news_sources = ['wallstreetjournal']
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Coordinates
first_video_point = (511, 851)  # Starting point (first video)
next_video_button = (1189, 527)  # Inside video, button to move to next

# ========== FUNCTIONS ==========

def scrape_current_video():
    """Scrape comments from the current video and return as a DataFrame."""
    # Move and click inside the comment area
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    # Scroll comments until end
    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(1.5)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)

    # Focus app window
    pyautogui.click(x=300, y=300)
    time.sleep(1)

    # Open DevTools (F12)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)

    # Focus Console
    pyautogui.click(x=496, y=840)
    time.sleep(2)

    # Inject JS code
    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""
    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')

    # Wait for JS execution
    time.sleep(5)

    # Read clipboard
    clipboard_content = pyperclip.paste()

    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()

def is_video_too_old(video_date_str):
    """Check if video date is before July 1st, 2024."""
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2024-"):
                date_part = video_date_str.split("2024-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2024
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2024 or (year == 2024 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

def restart_console():
    pyautogui.click(x=1539, y=132)  # Close DevTools
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=496, y=840)
    time.sleep(1)

# ========== MAIN LOGIC ==========

# Launch TikTok
os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    # Navigate back to search
    pyautogui.click(x=40, y=155)  # Back button
    time.sleep(2)
    pyautogui.click(x=141, y=160)  # Search bar
    time.sleep(2)
    pyautogui.hotkey('ctrl', 'a')
    pyautogui.press('backspace')
    pyautogui.write(source, interval=0.1)
    pyautogui.press('enter')
    time.sleep(4)
    pyautogui.click(x=659, y=210)  # Click first result
    time.sleep(4)

    all_comments = pd.DataFrame()
    video_counter = 0

    # Start by clicking the first video
    pyautogui.click(x=first_video_point[0], y=first_video_point[1])
    time.sleep(5)

    while True:
        df = scrape_current_video()

        if not df.empty:
            all_comments = pd.concat([all_comments, df], ignore_index=True)
            print(f"Scraped {len(df)} comments from video {video_counter + 1}")
            sample_date = df['videoDate'].iloc[0]
            if is_video_too_old(sample_date):
                print(f"Video from {sample_date} is too old. Stopping for {source}.")

                # Close the tab (profile)
                pyautogui.click(x=45, y=81)
                time.sleep(2)

                break
            
        #restart devtools to prevent overload
        restart_console()


        # Click "Next Video"
        pyautogui.click(x=1826, y=793) #click outside of the console screen
        time.sleep(1) #don't rush
        pyautogui.click(x=next_video_button[0], y=next_video_button[1])
        time.sleep(5)
        video_counter += 1

    # Save all comments for this source
    all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
    print(f"Saved {len(all_comments)} total comments for {source}.")


<>:53: SyntaxWarning: invalid escape sequence '\d'
<>:53: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_20412\1352938770.py:53: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Scraped 13 comments from video 1
Scraped 54 comments from video 2
Scraped 2 comments from video 3
Error parsing JSON: Expecting value: line 2 column 1 (char 1)
Scraped 33 comments from video 5
Scraped 49 comments from video 6
Scraped 7 comments from video 7
Scraped 9 comments from video 8
Scraped 1 comments from video 9
Scraped 5 comments from video 10
Scraped 370 comments from video 11
Scraped 21 comments from video 12
Scraped 91 comments from video 13
Scraped 5 comments from video 14
Scraped 10 comments from video 15
Scraped 375 comments from video 16
Scraped 96 comments from video 17
Scraped 60 comments from video 18
Scraped 29 comments from video 19
Scraped 21 comments from video 20
Scraped 552 comments from video 21
Scraped 4 comments from video 22
Scraped 16 comments from video 23
Scraped 266 comments from video 24
Scraped 4 comments from video 25
Scraped 9 comments from video 26
Scraped 12 comments from video 27
Scraped 11 comments from video 28
Scraped 14 comments from video 29

C:\Users\zhout\AppData\Local\Temp\ipykernel_20412\1352938770.py:53: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [ ]:
#fully working recovery video version. If it fails, click on the following one

import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json
import sys

# ========== SETTINGS ==========

#news_sources = ['nytimes', 'washingtonpost', 'usatoday', 'wallstreetjournal', 'latimes', 'cnn', 'foxnews', 'msnbc', 'abcnews', 'cbsnews', 'nbcnews', 'pbsnews', 'huffpost']
news_sources = ['wallstreetjournal']
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Coordinates
first_video_point = (511, 851)  # Starting point (first video)
next_video_button = (1189, 527)  # Inside video, button to move to next

# ========== FUNCTIONS ==========

def scrape_current_video():
    """Scrape comments from the current video and return as a DataFrame."""
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(1.5)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)
    pyautogui.click(x=300, y=300)
    time.sleep(1)

    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=496, y=840)
    time.sleep(2)

    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""
    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')
    time.sleep(5)

    clipboard_content = pyperclip.paste()

    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()

def is_video_too_old(video_date_str):
    """Check if video date is before July 1st, 2024."""
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2024-"):
                date_part = video_date_str.split("2024-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2024
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2024 or (year == 2024 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

def restart_console():
    """Restart DevTools console."""
    pyautogui.click(x=1539, y=132)  # Close DevTools
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=496, y=840)
    time.sleep(1)

# ========== MAIN LOGIC ==========

# Launch TikTok
os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    try:
        pyautogui.click(x=40, y=155)
        time.sleep(2)
        pyautogui.click(x=141, y=160)
        time.sleep(2)
        pyautogui.hotkey('ctrl', 'a')
        pyautogui.press('backspace')
        pyautogui.write(source, interval=0.1)
        pyautogui.press('enter')
        time.sleep(4)
        pyautogui.click(x=659, y=210)
        time.sleep(4)

        all_comments = pd.DataFrame()
        video_counter = 0

        pyautogui.click(x=first_video_point[0], y=first_video_point[1])
        time.sleep(5)

        for i in range(10):
            print(f"Preloading video {i+2} of 365...")
            pyautogui.click(x=next_video_button[0], y=next_video_button[1])
            time.sleep(1)

        while True:
            try:
                df = scrape_current_video()

                if not df.empty:
                    all_comments = pd.concat([all_comments, df], ignore_index=True)
                    print(f"Scraped {len(df)} comments from video {video_counter + 1}")

                    # ✅✅✅ AUTOSAVE RIGHT AFTER EACH VIDEO
                    temp_filename = f"tiktok_comments_scraped_{source}_autosave.csv"
                    all_comments.to_csv(temp_filename, index=False)
                    print(f"Auto-saved progress to {temp_filename}")

                    sample_date = df['videoDate'].iloc[0]
                    if is_video_too_old(sample_date):
                        print(f"Video from {sample_date} is too old. Stopping for {source}.")

                        pyautogui.click(x=45, y=81)
                        time.sleep(2)
                        break

                restart_console()

                pyautogui.click(x=1826, y=793)
                time.sleep(1)
                pyautogui.click(x=next_video_button[0], y=next_video_button[1])
                time.sleep(5)
                video_counter += 1

            except KeyboardInterrupt:
                print("Manual interrupt detected. Saving and exiting...")
                all_comments.to_csv(f"tiktok_comments_scraped_{source}_interrupted.csv", index=False)
                sys.exit(1)

            except Exception as e:
                print(f"Scraping error: {e}")
                all_comments.to_csv(f"tiktok_comments_scraped_{source}_error.csv", index=False)
                sys.exit(1)

        all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
        print(f"Saved {len(all_comments)} total comments for {source}.")

    except KeyboardInterrupt:
        print("Manual interrupt detected. Saving and exiting...")
        all_comments.to_csv(f"tiktok_comments_scraped_{source}_interrupted.csv", index=False)
        sys.exit(1)

    except Exception as e:
        print(f"Major error with {source}: {e}")
        all_comments.to_csv(f"tiktok_comments_scraped_{source}_major_error.csv", index=False)
        sys.exit(1)


<>:44: SyntaxWarning: invalid escape sequence '\d'
<>:44: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_17776\3073286735.py:44: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Preloading video 2 of 365...
Preloading video 3 of 365...
Preloading video 4 of 365...
Preloading video 5 of 365...
Preloading video 6 of 365...
Preloading video 7 of 365...
Preloading video 8 of 365...
Preloading video 9 of 365...
Preloading video 10 of 365...
Preloading video 11 of 365...
Manual interrupt detected. Saving and exiting...


SystemExit: 1

C:\Users\zhout\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
C:\Users\zhout\AppData\Local\Temp\ipykernel_17776\3073286735.py:44: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


In [10]:
import pyautogui
import time

time.sleep(5)
print(pyautogui.position())


Point(x=763, y=169)


In [ ]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json
import sys

# ========== SETTINGS ==========

#news_sources = ['nytimes', 'washingtonpost', 'usatoday', 'wallstreetjournal', 'latimes', 'cnn', 'foxnews', 'msnbc', 'abcnews', 'cbsnews', 'nbcnews', 'pbsnews', 'huffpost']
news_sources = ['thefemalelead']
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Coordinates
first_video_point = (511, 851)  # Starting point (first video)
next_video_button = (1189, 527)  # Inside video, button to move to next

# ========== FUNCTIONS ==========

def scrape_current_video():
    """Scrape comments from the current video and return as a DataFrame."""
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(1.5)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)

    # Focus app window (same)
    pyautogui.click(x=300, y=300)
    time.sleep(1)

    # ✅ Try to open console with F12 first
    pyautogui.hotkey('fn', 'f12')
    time.sleep(3)

    # ✅ Check if Console opened correctly by trying to click at console area
    pyautogui.moveTo(496, 840)
    pyautogui.click()
    time.sleep(2)

    # ✅ Quick test: is console open? (optional: skip or assume)
    # If not responding (console not open), fallback
    # Let's assume after clicking, if it didn't open properly, fallback

    # Attempt to paste something into console to test if it's active
    pyperclip.copy('console.log("test");')
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(0.5)
    pyautogui.press('enter')
    time.sleep(2)

    # Copy clipboard again to see if paste worked
    clipboard_check = pyperclip.paste()

    if 'test' not in clipboard_check:
        print("Console didn't open correctly. Falling back to alternative method...")

        # ✅ Alternative method: right-click -> inspect -> console
        pyautogui.moveTo(1243, 455)    # Move to video area
        pyautogui.rightClick()         # Right-click
        time.sleep(1)
        pyautogui.moveTo(1324, 946)    # Move to "Inspect"
        pyautogui.click()
        time.sleep(4)                  # Wait longer for DevTools
        pyautogui.moveTo(763, 169)     # Move to "Console" tab
        pyautogui.click()
        time.sleep(2)

    # ✅ Now inject actual JS
    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""
    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')
    time.sleep(5)

    clipboard_content = pyperclip.paste()

    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()


def is_video_too_old(video_date_str):
    """Check if video date is before July 1st, 2024."""
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2024-"):
                date_part = video_date_str.split("2024-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2024
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2024 or (year == 2024 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

def restart_console():
    """Restart DevTools console."""
    pyautogui.click(x=1539, y=132)  # Close DevTools
    time.sleep(1)
    pyautogui.hotkey('fn', 'f12')
    time.sleep(2)
    pyautogui.click(x=496, y=840)
    time.sleep(1)

# ========== MAIN LOGIC ==========

# Launch TikTok
os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    try:
        pyautogui.click(x=40, y=155)
        time.sleep(2)
        pyautogui.click(x=141, y=160)
        time.sleep(2)
        pyautogui.hotkey('ctrl', 'a')
        pyautogui.press('backspace')
        pyautogui.write(source, interval=0.1)
        pyautogui.press('enter')
        time.sleep(4)
        pyautogui.click(x=659, y=210)
        time.sleep(4)

        all_comments = pd.DataFrame()
        video_counter = 0

        pyautogui.click(x=first_video_point[0], y=first_video_point[1])
        time.sleep(5)

        for i in range(1):
            print(f"Preloading video {i+2} of 365...")
            pyautogui.click(x=next_video_button[0], y=next_video_button[1])
            time.sleep(1)

        while True:
            try:
                df = scrape_current_video()

                if not df.empty:
                    all_comments = pd.concat([all_comments, df], ignore_index=True)
                    print(f"Scraped {len(df)} comments from video {video_counter + 1}")

                    # ✅✅✅ AUTOSAVE RIGHT AFTER EACH VIDEO
                    temp_filename = f"tiktok_comments_scraped_{source}_autosave.csv"
                    all_comments.to_csv(temp_filename, index=False)
                    print(f"Auto-saved progress to {temp_filename}")

                    sample_date = df['videoDate'].iloc[0]
                    if is_video_too_old(sample_date):
                        print(f"Video from {sample_date} is too old. Stopping for {source}.")

                        pyautogui.click(x=45, y=81)
                        time.sleep(2)
                        break

                restart_console()

                pyautogui.click(x=1826, y=793)
                time.sleep(1)
                pyautogui.click(x=next_video_button[0], y=next_video_button[1])
                time.sleep(5)
                video_counter += 1

            except KeyboardInterrupt:
                print("Manual interrupt detected. Saving and exiting...")
                all_comments.to_csv(f"tiktok_comments_scraped_{source}_interrupted.csv", index=False)
                sys.exit(1)

            except Exception as e:
                print(f"Scraping error: {e}")
                all_comments.to_csv(f"tiktok_comments_scraped_{source}_error.csv", index=False)
                sys.exit(1)

        all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
        print(f"Saved {len(all_comments)} total comments for {source}.")

    except KeyboardInterrupt:
        print("Manual interrupt detected. Saving and exiting...")
        all_comments.to_csv(f"tiktok_comments_scraped_{source}_interrupted.csv", index=False)
        sys.exit(1)

    except Exception as e:
        print(f"Major error with {source}: {e}")
        all_comments.to_csv(f"tiktok_comments_scraped_{source}_major_error.csv", index=False)
        sys.exit(1)


<>:79: SyntaxWarning: invalid escape sequence '\d'
<>:79: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_17776\3900760761.py:79: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Preloading video 2 of 365...
Scraped 227 comments from video 1
Auto-saved progress to tiktok_comments_scraped_thefemalelead_autosave.csv
Video from 2021-12-15 is too old. Stopping for thefemalelead.
Saved 227 total comments for thefemalelead.


In [18]:
import os
import time
import pyautogui
import pyperclip
import pandas as pd
import json
import sys

# ========== SETTINGS ==========

news_sources = ['thefemalelead']
shortcut_path = r"C:\Users\zhout\Desktop\TikTok.lnk"

# Coordinates
first_video_point = (511, 851)  # Starting point (first video)
next_video_button = (1189, 527)  # Inside video, button to move to next

# ========== FUNCTIONS ==========

def close_devtools():
    """Close DevTools if it's open."""
    pyautogui.click(x=1539, y=132)  # 'X' close button for DevTools
    time.sleep(1)

def open_devtools_console():
    """Open DevTools console, with fallback if F12 fails."""
    pyautogui.hotkey('fn', 'f12')
    time.sleep(4)

    pyautogui.moveTo(496, 840)
    pyautogui.click()
    time.sleep(2)

    pyperclip.copy('console.log("test_devtools");')
    pyautogui.hotkey('ctrl', 'v')
    pyautogui.press('enter')
    time.sleep(2)

    clipboard_content = pyperclip.paste()

    if 'test_devtools' not in clipboard_content:
        print("DevTools console did not respond. Falling back to manual right-click method...")
        pyautogui.moveTo(1243, 455)
        pyautogui.rightClick()
        time.sleep(1)
        pyautogui.moveTo(1324, 946)
        pyautogui.click()
        time.sleep(4)
        pyautogui.moveTo(763, 169)
        pyautogui.click()
        time.sleep(2)
    else:
        print("DevTools opened successfully.")

def scrape_current_video():
    """Scrape comments from the current video and return as a DataFrame."""
    pyautogui.click(x=1605, y=591)
    time.sleep(1)

    previous = pyautogui.screenshot(region=(1200, 600, 100, 100))
    while True:
        pyautogui.scroll(-8000)
        time.sleep(1.5)
        current = pyautogui.screenshot(region=(1200, 600, 100, 100))
        if current == previous:
            break
        previous = current

    time.sleep(2)

    pyautogui.click(x=300, y=300)  # Focus app
    time.sleep(1)

    open_devtools_console()

    js_code = """
const comments = [];
let videoDate = "";
let username = "";
let caption = "";
let videoLink = "";

const usernameLink = document.querySelector('a[href^="/@"] span[data-e2e="browse-username"] span');
if (usernameLink) username = usernameLink.innerText.trim();

const captionBlock = document.querySelector('[data-e2e="browse-video-desc"]');
if (captionBlock) caption = captionBlock.innerText.trim();

const videoLinkBlock = document.querySelector('[data-e2e="browse-video-link"]');
if (videoLinkBlock) videoLink = videoLinkBlock.innerText.trim();

const nicknameBlock = document.querySelector('[data-e2e="browser-nickname"]');
if (nicknameBlock) {
    const spans = nicknameBlock.querySelectorAll('span');
    spans.forEach(span => {
        const text = span.innerText.trim();
        if (text.match(/^\d{4}-\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d{1,2}-\d{1,2}/)) videoDate = text;
        else if (text.match(/^\d+[mhdy]\\s*ago$/i)) videoDate = text;
    });
}

document.querySelectorAll('[class*="DivCommentContentContainer"]').forEach(commentEl => {
    const comment = commentEl.querySelector('p[data-e2e="comment-level-1"] span')?.innerText ?? "";
    const commentDate = commentEl.querySelector('[data-e2e^="comment-time"]')?.innerText ?? "";
    const likes = commentEl.querySelector('[data-e2e="comment-like-count"]')?.innerText ?? "";
    comments.push({ comment, commentDate, likes, videoDate, username, caption, videoLink });
});

console.log(comments);
copy(comments);
"""
    pyperclip.copy(js_code)
    pyautogui.hotkey('ctrl', 'v')
    time.sleep(1)
    pyautogui.press('enter')
    time.sleep(5)

    clipboard_content = pyperclip.paste()

    try:
        comments = json.loads(clipboard_content)
        df = pd.DataFrame(comments)
        return df
    except Exception as e:
        print("Error parsing JSON:", e)
        return pd.DataFrame()

def is_video_too_old(video_date_str):
    """Check if video date is before July 1st, 2024."""
    try:
        if "-" in video_date_str:
            if video_date_str.count("-") == 2:
                year, month, day = map(int, video_date_str.split("-"))
            elif video_date_str.startswith("2020-"):
                date_part = video_date_str.split("2020-")[1]
                month, day = map(int, date_part.split("-"))
                year = 2020
            else:
                month, day = map(int, video_date_str.split("-"))
                year = 2025
            if year < 2020 or (year == 2020 and (month < 7 or (month == 7 and day < 1))):
                return True
    except ValueError:
        pass
    return False

# ========== MAIN LOGIC ==========

# Launch TikTok
os.startfile(shortcut_path)
time.sleep(5)

for source in news_sources:
    try:
        pyautogui.click(x=40, y=155)
        time.sleep(2)
        pyautogui.click(x=141, y=160)
        time.sleep(2)
        pyautogui.hotkey('ctrl', 'a')
        pyautogui.press('backspace')
        pyautogui.write(source, interval=0.1)
        pyautogui.press('enter')
        time.sleep(4)
        pyautogui.click(x=659, y=210)
        time.sleep(4)

        all_comments = pd.DataFrame()
        video_counter = 0

        pyautogui.click(x=first_video_point[0], y=first_video_point[1])
        time.sleep(5)

        # Preload videos (if needed)
        for i in range(1):
            print(f"Preloading video {i+2} of 365...")
            pyautogui.click(x=next_video_button[0], y=next_video_button[1])
            time.sleep(1)

        while True:
            try:
                df = scrape_current_video()

                if not df.empty:
                    all_comments = pd.concat([all_comments, df], ignore_index=True)
                    print(f"Scraped {len(df)} comments from video {video_counter + 1}")

                    temp_filename = f"tiktok_comments_scraped_{source}_autosave.csv"
                    all_comments.to_csv(temp_filename, index=False)
                    print(f"Auto-saved progress to {temp_filename}")

                    sample_date = df['videoDate'].iloc[0]
                    if is_video_too_old(sample_date):
                        print(f"Video from {sample_date} is too old. Stopping for {source}.")

                        pyautogui.click(x=45, y=81)
                        time.sleep(2)
                        break

                # ✅ Correct order:

                close_devtools()  # 1. Close DevTools
                pyautogui.click(x=1826, y=793)  # 2. Click outside
                time.sleep(1)
                pyautogui.click(x=next_video_button[0], y=next_video_button[1])  # 3. Next Video
                time.sleep(5)
                open_devtools_console()  # 4. Re-open DevTools

                video_counter += 1

            except KeyboardInterrupt:
                print("Manual interrupt detected. Saving and exiting...")
                all_comments.to_csv(f"tiktok_comments_scraped_{source}_interrupted.csv", index=False)
                sys.exit(1)

            except Exception as e:
                print(f"Scraping error: {e}")
                all_comments.to_csv(f"tiktok_comments_scraped_{source}_error.csv", index=False)
                sys.exit(1)

        all_comments.to_csv(f"tiktok_comments_scraped_{source}.csv", index=False)
        print(f"Saved {len(all_comments)} total comments for {source}.")

    except KeyboardInterrupt:
        print("Manual interrupt detected. Saving and exiting...")
        all_comments.to_csv(f"tiktok_comments_scraped_{source}_interrupted.csv", index=False)
        sys.exit(1)

    except Exception as e:
        print(f"Major error with {source}: {e}")
        all_comments.to_csv(f"tiktok_comments_scraped_{source}_major_error.csv", index=False)
        sys.exit(1)


<>:76: SyntaxWarning: invalid escape sequence '\d'
<>:76: SyntaxWarning: invalid escape sequence '\d'
C:\Users\zhout\AppData\Local\Temp\ipykernel_17776\4009421768.py:76: SyntaxWarning: invalid escape sequence '\d'
  js_code = """


Preloading video 2 of 365...
DevTools opened successfully.
Scraped 227 comments from video 1
Auto-saved progress to tiktok_comments_scraped_thefemalelead_autosave.csv
DevTools opened successfully.
DevTools opened successfully.
Error parsing JSON: Expecting value: line 2 column 1 (char 1)
Manual interrupt detected. Saving and exiting...


SystemExit: 1

C:\Users\zhout\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
C:\Users\zhout\AppData\Local\Temp\ipykernel_17776\4009421768.py:76: SyntaxWarning: invalid escape sequence '\d'
  js_code = """
